# Re-extract subject headings 

In [2]:
import sys, re, os 
from bs4 import BeautifulSoup, SoupStrainer
from tqdm import tqdm  
import pandas as pd 
TCP = '/Users/amycweng/DH/TCP'
sys.path.append('../')

def findTextTCP(id):
    if re.match('B1|B4',id[0:2]):
        path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    else: 
        if f'{id}.P4.xml' in os.listdir(f'{TCP}/P1{id[0:2]}'):
            path = f'{TCP}/P1{id[0:2]}/{id}.P4.xml'
        elif f'{id}.P4.xml' in os.listdir(f'{TCP}/P2{id[0:2]}'): 
            path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    return path 

In [16]:
sermons = list(pd.read_csv("../assets/sermons.csv")['id'])
sermons.extend(list(pd.read_csv("../assets/sermons_missing.csv")["id"]))
tcpID_subjects = {}
progress_bar = tqdm(sermons)
for file in progress_bar:
    progress_bar.set_description(file)
    tcpID = file.split(".")[0]
    tcpID_subjects[tcpID] = []
    filepath = findTextTCP(tcpID)

    # read the input XML file 
    with open(filepath,'r',encoding='utf-8') as file: 
        data = file.read()
    # use soupstrainer to only parse the main body
    tag = SoupStrainer("KEYWORDS")
    soup = BeautifulSoup(data,features="xml",parse_only=tag)
    
    subjects = soup.find_all(['TERM'])
    for s in subjects: 
        tcpID_subjects[tcpID].append(s)

A28857: 100%|██████████| 5822/5822 [04:11<00:00, 23.16it/s]


In [21]:
tcpID_subjects = {k:[item.string for item in v] for k,v in tcpID_subjects.items()}

In [8]:
from tqdm import tqdm 
def correct_metadata_file(filepath):
    sermons = pd.read_csv(filepath)
    sermons = sermons.to_dict(orient='records')
    new_sermons = []
    for info_dict in tqdm(sermons):
        tcpID = info_dict['id']
        filepath = findTextTCP(tcpID)
        if re.search(f'TCP/P1',filepath): 
            phase = "1"
        else: phase = "2"
        info_dict['phase'] = phase
        new_sermons.append(info_dict)
    sermons = pd.DataFrame(new_sermons)
    sermons.to_csv(filepath,index=False)

In [ ]:
correct_metadata_file("../assets/sermons.csv")

In [ ]:
correct_metadata_file("../assets/sermons_missing.csv")

In [34]:
import pandas as pd 
from collections import Counter 
from tqdm import tqdm 

sermons = pd.read_csv("../assets/sermons.csv")
sermons = sermons.to_dict(orient='records')

missing = pd.read_csv("../assets/sermons_missing.csv")
missing = missing.to_dict(orient='records')
sermons.extend(missing)


all_subjects = []

for info_dict in tqdm(sermons):
    subjects = info_dict['subject_headings']
    if not isinstance(subjects,str): continue  
    
    for w in subjects.split("; "): 
        if w == "No Keywords": continue
        if w != 'O.T.' and w!= 'N.T.': 
            w = w.strip(".") 
        all_subjects.append(w)

processed = {}
for w, freq in sorted(dict(Counter(all_subjects)).items(),key=lambda x:x[1],reverse=True):
    w = re.sub(r"\s+"," ",w)
    w = re.sub(" -- Early works to 1800","",w)
    w = re.sub(r"\s+"," ",w)
    w = re.sub(" -- 16th century| -- 17th century","",w)
    w = re.sub(r"\s+"," ",w)
    w = re.sub(" -- Sermons","",w)
    w = re.sub(r"\s+"," ",w)
    w = re.sub("Great Britain -- History -- ","",w)
    w = re.sub(r"\s+"," ",w)
    if w not in processed: processed[w] = freq
    else: processed[w] += freq


len(processed), sorted(processed.items(),key=lambda x:x[1],reverse=True)[:100]

100%|██████████| 5858/5858 [00:00<00:00, 539601.90it/s]


(3979,
 [('Sermons, English', 4269),
  ('Funeral sermons', 570),
  ('Church of England', 313),
  ('Fast-day sermons', 247),
  ('Christian life', 144),
  ('Catholic Church -- Controversial literature', 72),
  ('Civil War, 1642-1649', 67),
  ('Sermons, American', 62),
  ("Lord's Supper", 55),
  ('Salvation', 54),
  ('Dissenters, Religious -- England', 48),
  ('Early works to 1800', 45),
  ('Sin', 45),
  ('Visitation sermons', 44),
  ('Charles -- I, -- King of England, 1600-1649', 44),
  ('Faith', 43),
  ('Repentance', 40),
  ('Charles -- II, -- King of England, 1630-1685', 36),
  ('Death', 35),
  ('Theology, Doctrinal', 33),
  ('Providence and government of God', 30),
  ('Grace (Theology)', 30),
  ('Jesus Christ', 26),
  ('Mary -- II, -- Queen of England, 1662-1694', 25),
  ('Justification', 25),
  ('Church and state -- England', 24),
  ('Conduct of life', 23),
  ('Judgment Day', 23),
  ('Charity', 22),
  ('Prayer', 22),
  ('Gunpowder Plot, 1605', 21),
  ('Charles II, 1660-1685', 21),
  